Импорт необходимых модулей

In [ ]:
# main.ipynb

# Импорт необходимых модулей
import os
import json
import pandas as pd
from datetime import datetime
from models.language_models import get_model_instance
from questionnaires import BFI44  # Или другие опросники
from utils import save_results
from text_generation import generate_texts
from text_analysis import analyze_texts
from plotting import plot_confusion_matrices
import logging

# Настройка логирования
logging.basicConfig(
    filename='experiment.log',
    level=logging.INFO,
    format='%(asctime)s %(levelname)s:%(message)s'
)
logger = logging.getLogger(__name__)
